# GDS for Snowflake: Entity Resolution & Fraud Labels

In [ ]:
# Snowpark for Python
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.functions import udf
import snowflake.snowpark.functions as F

from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
%%sql
USE ROLE accountadmin;
USE DATABASE p2p_demo;
USE SCHEMA public;

In [ ]:
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

In [ ]:
%%sql
SELECT * from p2p_users

In [ ]:
%%sql
SELECT * FROM resolved_p2p_users;

## Combining Graph Features & Additional Fraud Risk Labeling
Using another view

In [1]:
%%sql
CREATE OR REPLACE VIEW user_features AS
SELECT p2p_users.* RENAME nodeId AS user_id,
    gds_features.wcc_id,
    gds_features.has_fraud_flag AS fraud_risk,
    gds_features.user_count AS community_size,
    TO_NUMBER(gds_features.user_count > 1) AS part_of_community,
    gds_features.transaction_pagerank
-- join users to gds features
FROM p2p_users JOIN (
    -- join resolved user view with gds algo results on wcc_id
    SELECT resolved_p2p_users.wcc_id, 
        resolved_p2p_users.user_count, 
        resolved_p2p_users.has_fraud_flag,
        gds_algo_results.transaction_pagerank,
        gds_algo_results.nodeId
    FROM resolved_p2p_users JOIN (
        -- join gds output tables on node
        SELECT p2p_users_wcc.nodeId, 
             p2p_users_wcc.wcc_id, 
            pagerank.transaction_pagerank
        FROM p2p_users_wcc
        JOIN (
            SELECT p2p_users_pagerank.nodeId, 
                p2p_users_pagerank.score AS transaction_pagerank
            FROM p2p_users_pagerank
        ) pagerank ON  p2p_users_wcc.nodeId = pagerank.nodeId
    ) gds_algo_results ON gds_algo_results.wcc_id = resolved_p2p_users.wcc_id
) gds_features ON user_id = gds_features.nodeId;
SELECT * FROM user_features

UsageError: Cell magic `%%sql` not found.


## Additional Fraud Risk Labels from WCC Entity Resolution

In [ ]:
user_feat_df = feature_table.to_pandas()
flagged_num = user_feat_df.FRAUD_TRANSFER_FLAG.sum()
fraud_risk_num = user_feat_df.FRAUD_RISK.sum()
print(f'# of original flagged accounts: {flagged_num:.0f}')
print(f'# of newly labeled fraud risk accounts from WCC: {fraud_risk_num:.0f}')
print(f'{(fraud_risk_num - flagged_num):.0f} new accounts identified, a {100*(fraud_risk_num - flagged_num)/flagged_num:.1f}% increase')